In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [3]:
import astroid
from pathlib import Path

In [9]:
base_path = Path("/workspaces/SebastianBot/cloud/functions")

In [10]:
file = Path("/workspaces/SebastianBot/cloud/functions/side_effects/complete_task/models.py")

In [53]:
from cloud.helper.event_grid import EventGridModel
import importlib


def extract_event_grid_models() -> list[str]:
    base_name = EventGridModel.__name__
    class_names = []
    def extract_inherited_classnames(node: astroid.nodes.ClassDef, base_name:str) -> str|None:
        base_names = [b.name for b in node.bases]
        if base_name in base_names:
            return node.name
        return None

    for file in base_path.rglob("*.py"):
        module = astroid.parse(file.read_text())
        for node in module.body:
            if not isinstance(node, astroid.nodes.ClassDef):
                continue

            if not (name := extract_inherited_classnames(node, base_name)):
                continue

            # Convert file path to module path
            relative_path = file.relative_to(Path("/workspaces/SebastianBot"))
            module_path = str(relative_path.with_suffix("")).replace("/", ".")
            
            # Dynamically import the module and get the class
            imported_module = importlib.import_module(module_path)
            class_type = getattr(imported_module, name)
            class_names.append(class_type.env_name())

    return class_names

In [54]:
extract_event_grid_models()

['AllActorEventGrid', 'CompleteTaskEventGrid', 'CreateTaskEventGrid']